In [1]:
import pathlib
import cv2
import numpy as np
import tqdm
import shutil
import random

In [2]:
def create_clean_folder(path):
    path = pathlib.Path(path)
    path.mkdir(parents=True, exist_ok=True)
    shutil.rmtree(path)
    path.mkdir(parents=True, exist_ok=True)

data_path = pathlib.Path('../../data/')
inria_path = data_path / 'INRIA'
output_path = inria_path / 'output'

train_path = output_path / 'train'
train_images_path = train_path / 'image'
train_labels_path = train_path / 'label'

val_path = output_path / 'val'
val_images_path = val_path / 'image'
val_labels_path = val_path / 'label'

test_path = output_path / 'test'

train_aug_path = output_path / 'train_aug'
train_aug_images_path = train_aug_path / 'image'
create_clean_folder(train_aug_images_path)
train_aug_labels_path = train_aug_path / 'label'
create_clean_folder(train_aug_labels_path)

val_aug_path = output_path / 'val_aug'
val_aug_images_path = val_aug_path / 'image'
create_clean_folder(val_aug_images_path)
val_aug_labels_path = val_aug_path / 'label'
create_clean_folder(val_aug_labels_path)

## Data augmentation

Transformations performed:

- Multiplying the sample data by a factor defined by AUGMENT_FACTOR
- Randomly rotate by 0, 90, 180 or 270 degrees
- Randomly perform an average filter
- Randomly flip horizontally
- Randomly flip vertically

In [3]:
def multiply_list_float(l, factor):
    factor_int = int(factor)
    out = l * factor_int
    diff = factor - factor_int
    diff_len = int(diff * len(l))
    
    return out + l[:diff_len]

def process_pair(img, mask):
    # Randomly rotate by a multiply of 90, and correct pixels by corresponding value
    rotations = [None, cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_180, cv2.ROTATE_90_COUNTERCLOCKWISE]
    rotation = random.choice(rotations)
    
    if rotation:
        img = cv2.rotate(img, rotation)
        mask = cv2.rotate(mask, rotation)
    
    # Random average filtering every other image
    avg_filter = random.uniform(0, 1)
    if avg_filter > 0.5:
        kernel = np.ones((5,5),np.float32) / random.randrange(15,30)
        img = cv2.filter2D(img,-1,kernel)
    
    # Flip vertically every other image
    flip_filter = random.uniform(0, 1)
    if flip_filter:
        mask = cv2.flip(mask, 0)
        img = cv2.flip(img, 0)
        
    # Flip horizontally every other image
    flip_filter = random.uniform(0, 1)
    if flip_filter:
        mask = cv2.flip(mask, 1)
        img = cv2.flip(img, 1)
    
    return img, mask

In [4]:
# Factor to augment the data, or how much to increase the original dataset
AUGMENT_FACTOR = 2
path_list = list(train_images_path.glob('*.png'))
path_list = multiply_list_float(path_list, AUGMENT_FACTOR)

random.shuffle(path_list)

for i, path in enumerate(tqdm.tqdm(path_list)):
    name = path.name
    img = cv2.imread(str(train_images_path / name))
    mask = cv2.imread(str(train_labels_path / name))
    
    img, mask = process_pair(img, mask)
    
    cv2.imwrite(str(train_aug_images_path / name.replace('.png', f'_t{i}.png')), img)
    cv2.imwrite(str(train_aug_labels_path / name.replace('.png', f'_t{i}.png')), mask)

100%|███████████████████████████████████████████████████████████| 23596/23596 [13:55<00:00, 28.25it/s]


In [5]:
# Factor to augment the data, or how much to increase the original dataset
AUGMENT_FACTOR = 1
path_list = list(val_images_path.glob('*.png'))
path_list = multiply_list_float(path_list, AUGMENT_FACTOR)

random.shuffle(path_list)

for i, path in enumerate(tqdm.tqdm(path_list)):
    name = path.name
    img = cv2.imread(str(val_images_path / name))
    mask = cv2.imread(str(val_labels_path / name))
    
    img, mask = process_pair(img, mask)
    
    cv2.imwrite(str(val_aug_images_path / name.replace('.png', f'_t{i}.png')), img)
    cv2.imwrite(str(val_aug_labels_path / name.replace('.png', f'_t{i}.png')), mask)

100%|█████████████████████████████████████████████████████████████| 3015/3015 [01:36<00:00, 31.35it/s]
